In [3]:
import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np
import sys
import os
import glob
from keras.preprocessing import image
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy
from scipy import misc
import scipy.io

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
def sort(path, im_type, backend):

    images = glob.glob1(path,"*." + str(im_type))
    im_type_length = len(im_type) + 1
    images = [element[:-im_type_length] for element in images]
    images.sort(key=int)
    images = [element + "." + str(im_type) for element in images]

    if backend == "keras" or backend == "Keras" or backend == "keras":
        images = [np.array(image.load_img(path + "/" + fname)) for fname in images]
    if backend == "opencv" or backend == "openCV" or backend == "OPENCV" or backend == "cv2":
        images = [np.array(cv2.imread(path + "/" + fname)) for fname in images]
    if backend == "PIL" or backend == "pil":
        images = [np.array(Image.open(path + "/" + fname)) for fname in images]
    return images

In [6]:
cropped_glauc = sort(path = "/Users/karimabedrabbo/Desktop/Ipython_Notebooks_Aug/cropped_glauc", im_type = "png", backend = "keras")
cropped_not_glauc = sort(path = "/Users/karimabedrabbo/Desktop/Ipython_Notebooks_Aug/cropped_not_glauc", im_type = "png", backend = "keras")

In [7]:
cropped_discs_glauc = sort(path = "/Users/karimabedrabbo/Desktop/Ipython_Notebooks_Aug/cropped_glauc_discs_png", im_type = "png", backend = "keras")
cropped_discs_not_glauc = sort(path = "/Users/karimabedrabbo/Desktop/Ipython_Notebooks_Aug/cropped_not_glauc_discs_png", im_type = "png", backend = "keras")

In [8]:
def save_images(aug_path, aug_imgs):
    for i in range(len(aug_imgs)):
        aug = misc.toimage(aug_imgs[i])
        aug.save(aug_path + "/" + str(i + 1) + ".png")

In [9]:
def sort_list(path_list, im_type, backend):
    final = []
    for path in path_list:
        final.append(sort(path, im_type, backend))
    return final

In [10]:
def multiple_image_numpy(imgs, rows, cols):
    total = len(imgs)
    new = np.ndarray((total, rows, cols, 3), dtype=np.uint8)

    for i in range(total):
        new[i] = imgs[i]
    
    return new


In [11]:
def rgb2gray(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

In [12]:
def get_rgb_medians(img, disc):
    img = cv2.GaussianBlur(img,(5,5),0)
    if (len(disc.shape) == 3):
        disc = rgb2gray(disc)
    img[disc == 0] = [0, 0, 0]
    image = Image.fromarray(img).convert('RGB')
    r = []
    g = []
    b = []
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            color = image.getpixel((i, j))
            if not color[0] == 0: r.append(color[0])
            if not color[0] == 0: g.append(color[1])
            if not color[0] == 0: b.append(color[2])
    r,g,b = np.median(r), np.median(g), np.median(b)
    return r,g,b

In [13]:
def run_augmentation(images):
    # Standard scenario: You have N RGB-images and additionally 21 heatmaps per image.
    # You want to augment each image and its heatmaps identically.
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)

    # Define our sequence of augmentation steps that will be applied to every image
    # All augmenters with per_channel=0.5 will sample one value _per image_
    # in 50% of all cases. In all other cases they will sample new values
    # _per channel_.
    seq = iaa.Sequential(
        [
            # apply the following augmenters to most images
            iaa.Fliplr(0.5), # horizontally flip 50% of all images
            sometimes(iaa.Affine(
                scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
                translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to +20 percent (per axis)
                rotate=(-179, 179), # rotate by -45 to +45 degrees
                #order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
                #cval=(0, 255), # if mode is constant, use a cval between 0 and 255
                #mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
            )),
            sometimes(iaa.GaussianBlur((0, 3.0)) # blur images with a sigma between 0 and 3.0
            ),
            sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)) # blur images with a sigma between 0 and 3.0
            ),
        ],
        random_order=True
    )
    images_aug = seq.augment_images(images)    
    return images_aug

In [14]:
def moder_aug(images):
    # Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
    # e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second image.
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)

    # Define our sequence of augmentation steps that will be applied to every image
    # All augmenters with per_channel=0.5 will sample one value _per image_
    # in 50% of all cases. In all other cases they will sample new values
    # _per channel_.
    seq = iaa.Sequential(
        [
            # apply the following augmenters to most images
            iaa.Fliplr(0.5), # horizontally flip 50% of all images
            iaa.OneOf([
                        iaa.GaussianBlur((0, 1.5)), # blur images with a sigma between 0 and 3.0
                        iaa.AverageBlur(k=(2, 3)), # blur image using local means with kernel sizes between 2 and 7
                        iaa.MedianBlur(k=(1, 5)), # blur image using local medians with kernel sizes between 2 and 7
                    ]),
            iaa.Add((-190, -170)), # change brightness of images (by -10 to 10 of original value)
            #iaa.Flipud(0.2), # vertically flip 20% of all images
            # crop images by -5% to 10% of their height/width
            sometimes(iaa.CropAndPad(
                percent=(-0.02, 0.05),
                pad_mode='constant',
                pad_cval=(0, 0)
            )),
            sometimes(iaa.Affine(
                scale={"x": (0.95, 1.05), "y": (0.95, 1.05)}, # scale images to 80-120% of their size, individually per axis
                translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, # translate by -20 to +20 percent (per axis)
                rotate=(-15, 15), # rotate by -45 to +45 degrees
                #shear=(-16, 16), # shear by -16 to +16 degrees
                order=[0], # use nearest neighbour or bilinear interpolation (fast)
                cval=(0, 0), # if mode is constant, use a cval between 0 and 255
                mode='constant' # use any of scikit-image's warping modes (see 2nd image from the top for examples)
            )),
            # execute 0 to 5 of the following (less important) augmenters per image
            # don't execute all of them, as that would often be way too strong
            iaa.SomeOf((0, 1),
                [
                    #sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))), # convert images into their superpixel representation
                    #iaa.AddToHueAndSaturation((-1, 1)), # change hue and saturation
                    #iaa.DirectedEdgeDetect(alpha=(0.1, 0.5), direction=(0.0, 1.0)),
                    #iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), # add gaussian noise to images
                    #iaa.OneOf([
                    #    iaa.Dropout((0.01, 0.1), per_channel=0.5), # randomly remove up to 10% of the pixels
                    #    iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
                    #]),
                    # either change the brightness of the whole image (sometimes
                    # per channel) or change the brightness of subareas

                    #iaa.ContrastNormalization((0.1, 0.3)), # improve or worsen the contrast
                    #sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))), # sometimes move parts of the image around
                    #sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
                ],
                random_order=True
            )
        ],
        random_order=True
    )

    images_aug = seq.augment_images(images)
    return images_aug

In [15]:
def moder_aug(images):
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)
    
    print(type(iaa))
    seq = iaa.Sequential(
        [
            #iaa.Add((-150, -130)),
            #iaa.WithChannels(channels=[0],children=iaa.Add((-70, -60))),
            iaa.Multiply((0.3, 0.5)),
            iaa.Fliplr(0.5), # horizontally flip 50% of all images
            #iaa.OneOf([
            #            iaa.GaussianBlur((0, 1.5)), # blur images with a sigma between 0 and 3.0
            #            iaa.AverageBlur(k=(2, 3)), # blur image using local means with kernel sizes between 2 and 7
            #            iaa.MedianBlur(k=(1, 5)), # blur image using local medians with kernel sizes between 2 and 7
            #        ]),
            #sometimes(iaa.Affine(
            #    scale={"x": (0.95, 1.05), "y": (0.95, 1.05)}, # scale images to 80-120% of their size, individually per axis
            #    translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, # translate by -20 to +20 percent (per axis)
            #    rotate=(-15, 15), # rotate by -45 to +45 degrees
            #    order=[0], # use nearest neighbour or bilinear interpolation (fast)
            #    cval=(0, 0), # if mode is constant, use a cval between 0 and 255
            #    mode='constant' # use any of scikit-image's warping modes (see 2nd image from the top for examples)
            #))
        ],
        random_order=False
    )

    images_aug = seq.augment_images(images)
    return images_aug

In [16]:
def moder_aug_single(image, disc, sigma):
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)
    r,g,b = get_rgb_medians(image, disc)
    
    rc, bc, gc = 103 - r, 60 - g,38 - b
    
    seq = iaa.Sequential(
        [
            #iaa.Add((-150, -130)),
            iaa.WithChannels(channels=[0],children=iaa.Add((rc,rc))),
            iaa.WithChannels(channels=[1],children=iaa.Add((gc,gc))),
            iaa.WithChannels(channels=[2],children=iaa.Add((bc,bc))),
            #iaa.WithChannels(channels=[0],children=iaa.Multiply(((r-sigma)/r, (r-sigma)/r))),
            #iaa.WithChannels(channels=[1],children=iaa.Multiply(((g-(sigma*.573))/g, (g-(sigma*.573))/g))),
            #iaa.WithChannels(channels=[2],children=iaa.Multiply(((b-(sigma*.246) )/b, (b-(sigma*.246))/b))),
            #iaa.ContrastNormalization((1.0004, 1.0004)),
            #iaa.Multiply((0.8, 0.9)),
            #iaa.Fliplr(0.5), # horizontally flip 50% of all images
            #iaa.OneOf([
            #            iaa.GaussianBlur((0, 1.5)), # blur images with a sigma between 0 and 3.0
            #            iaa.AverageBlur(k=(2, 3)), # blur image using local means with kernel sizes between 2 and 7
            #            iaa.MedianBlur(k=(1, 5)), # blur image using local medians with kernel sizes between 2 and 7
            #        ]),
            #sometimes(iaa.Affine(
            #    scale={"x": (0.95, 1.05), "y": (0.95, 1.05)}, # scale images to 80-120% of their size, individually per axis
            #    translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, # translate by -20 to +20 percent (per axis)
            #    rotate=(-15, 15), # rotate by -45 to +45 degrees
            #    order=[0], # use nearest neighbour or bilinear interpolation (fast)
            #    cval=(0, 0), # if mode is constant, use a cval between 0 and 255
            #    mode='constant' # use any of scikit-image's warping modes (see 2nd image from the top for examples)
            #))
        ],
        random_order=False
    )
    image = seq.augment_image(image)
    r,g,b = get_rgb_medians(image, disc)
    return image

In [17]:
moderaug_cropped_not_glauc = []
for i in range(20):
    moderaug_cropped_not_glauc.append(moder_aug_single(cropped_not_glauc[i],cropped_discs_not_glauc[i], 2.))
    print(i)
moderaug_cropped_not_glauc = np.array(moderaug_cropped_not_glauc) 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [18]:
save_images("/Users/karimabedrabbo/Desktop/Ipython_Notebooks_Aug/moderaug_cropped_not_glauc",moderaug_cropped_not_glauc)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [82]:
def heavy_aug(images):
    # Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
    # e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second image.
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)

    # Define our sequence of augmentation steps that will be applied to every image
    # All augmenters with per_channel=0.5 will sample one value _per image_
    # in 50% of all cases. In all other cases they will sample new values
    # _per channel_.
    seq = iaa.Sequential(
        [
            # apply the following augmenters to most images
            iaa.Fliplr(0.5), # horizontally flip 50% of all images
            iaa.Flipud(0.2), # vertically flip 20% of all images
            # crop images by -5% to 10% of their height/width
            sometimes(iaa.CropAndPad(
                percent=(-0.05, 0.1),
                pad_mode=ia.ALL,
                pad_cval=(0, 255)
            )),
            sometimes(iaa.Affine(
                scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
                translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to +20 percent (per axis)
                rotate=(-45, 45), # rotate by -45 to +45 degrees
                shear=(-16, 16), # shear by -16 to +16 degrees
                order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
                cval=(0, 255), # if mode is constant, use a cval between 0 and 255
                mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
            )),
            # execute 0 to 5 of the following (less important) augmenters per image
            # don't execute all of them, as that would often be way too strong
            iaa.SomeOf((0, 5),
                [
                    sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))), # convert images into their superpixel representation
                    iaa.OneOf([
                        iaa.GaussianBlur((0, 3.0)), # blur images with a sigma between 0 and 3.0
                        iaa.AverageBlur(k=(2, 7)), # blur image using local means with kernel sizes between 2 and 7
                        iaa.MedianBlur(k=(3, 11)), # blur image using local medians with kernel sizes between 2 and 7
                    ]),
                    iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), # sharpen images
                    iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), # emboss images
                    # search either for all edges or for directed edges,
                    # blend the result with the original image using a blobby mask
                    iaa.SimplexNoiseAlpha(iaa.OneOf([
                        iaa.EdgeDetect(alpha=(0.5, 1.0)),
                        iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
                    ])),
                    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), # add gaussian noise to images
                    iaa.OneOf([
                        iaa.Dropout((0.01, 0.1), per_channel=0.5), # randomly remove up to 10% of the pixels
                        iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
                    ]),
                    iaa.Invert(0.05, per_channel=True), # invert color channels
                    iaa.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
                    iaa.AddToHueAndSaturation((-20, 20)), # change hue and saturation
                    # either change the brightness of the whole image (sometimes
                    # per channel) or change the brightness of subareas
                    iaa.OneOf([
                        iaa.Multiply((0.5, 1.5), per_channel=0.5),
                        iaa.FrequencyNoiseAlpha(
                            exponent=(-4, 0),
                            first=iaa.Multiply((0.5, 1.5), per_channel=True),
                            second=iaa.ContrastNormalization((0.5, 2.0))
                        )
                    ]),
                    iaa.ContrastNormalization((0.5, 2.0), per_channel=0.5), # improve or worsen the contrast
                    iaa.Grayscale(alpha=(0.0, 1.0)),
                    sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), # move pixels locally around (with random strengths)
                    sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))), # sometimes move parts of the image around
                    sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
                ],
                random_order=True
            )
        ],
        random_order=True
    )

    images_aug = seq.augment_images(images)
    return images_aug

In [90]:
def save_images(aug_path, aug_imgs):
    for i in range(len(aug_imgs)):
        aug = misc.toimage(aug_imgs[i])
        aug.save(aug_path + "/" + str(i + 1) + ".png")

In [12]:
lightaug_cropped_glauc = run_augmentation(cropped_glauc)
lightaug_cropped_not_glauc = run_augmentation(cropped_not_glauc)

In [59]:
save_images("/Users/karimabedrabbo/Desktop/Ipython_Notebooks_Aug/lightaug_cropped_glauc",lightaug_cropped_glauc)
save_images("/Users/karimabedrabbo/Desktop/Ipython_Notebooks_Aug/lightaug_cropped_not_glauc",lightaug_cropped_not_glauc)

NameError: name 'lightaug_cropped_glauc' is not defined

In [91]:
moderaug_cropped_glauc = moder_aug(cropped_glauc)
moderaug_cropped_not_glauc = moder_aug(cropped_not_glauc)

In [92]:
save_images("/Users/karimabedrabbo/Desktop/Ipython_Notebooks_Aug/moderaug_cropped_glauc",moderaug_cropped_glauc)
save_images("/Users/karimabedrabbo/Desktop/Ipython_Notebooks_Aug/moderaug_cropped_not_glauc",moderaug_cropped_not_glauc)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
heavyaug_cropped_glauc = heavy_aug(cropped_glauc)
heavyaug_cropped_not_glauc = heavy_aug(cropped_not_glauc)

In [15]:
save_images("/Users/karimabedrabbo/Desktop/Ipython_Notebooks_Aug/heavyaug_cropped_glauc",heavyaug_cropped_glauc)
save_images("/Users/karimabedrabbo/Desktop/Ipython_Notebooks_Aug/heavyaug_cropped_not_glauc",heavyaug_cropped_not_glauc)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  This is separate from the ipykernel package so we can avoid doing imports until
